--- Day 3: Binary Diagnostic ---

The submarine has been making some odd creaking noises, so you ask it to produce a diagnostic report just in case.

The diagnostic report (your puzzle input) consists of a list of binary numbers which, when decoded properly, can tell you many useful things about the conditions of the submarine. The first parameter to check is the **power consumption**.

You need to use the binary numbers in the diagnostic report to generate two new binary numbers (called the **gamma rate** and the **epsilon rate**). The power consumption can then be found by multiplying the gamma rate by the epsilon rate.

Each bit in the gamma rate can be determined by finding the **most common bit in the corresponding position** of all numbers in the diagnostic report. For example, given the following diagnostic report:

```
00100
11110
10110
10111
10101
01111
00111
11100
10000
11001
00010
01010
```

Considering only the first bit of each number, there are five `0` bits and seven `1` bits. Since the most common bit is `1`, the first bit of the gamma rate is `1`.

The most common second bit of the numbers in the diagnostic report is `0`, so the second bit of the gamma rate is `0`.

The most common value of the third, fourth, and fifth bits are `1`, `1`, and `0`, respectively, and so the final three bits of the gamma rate are `110`.

So, the gamma rate is the binary number `10110`, or `22` in decimal.

The epsilon rate is calculated in a similar way; rather than use the most common bit, the least common bit from each position is used. So, the epsilon rate is `01001`, or `9` in decimal. Multiplying the gamma rate (`22`) by the epsilon rate (`9`) produces the power consumption, `198`.

Use the binary numbers in your diagnostic report to calculate the gamma rate and epsilon rate, then multiply them together. **What is the power consumption of the submarine**? (Be sure to represent your answer in decimal, not binary.)

## Solution

I'm assuming that all the bit lengths are the same, but it's good to check.  So I'll write a quick function to add all the bit lengths to a set to remove any duplicate lengths.  If the set returned has more than one length then that might change the method I need to use to determine the most common bits.

In [1]:
def get_bit_length(filename: str) -> set:
    bit_lengths = set()
    with open(filename, 'r') as f:
        for line in f:
            bit_lengths.add(len(line.strip()))
    return bit_lengths

sample_lengths = get_bit_length('sample.txt')
input_lengths = get_bit_length('input.txt')
print(f"Sample bit lengths: {sample_lengths}\nInput Lengths: {input_lengths}\n")

Sample bit lengths: {5}
Input Lengths: {12}



Great!  The bit lengths are all consistent per file, so that will make calculating the most common bits per file easier.  However, since the sample has a different bit length than the input, I'll want to account for variability per file.  In order to take advantage of the file object iterator, I can use a matrix to store a counter for each bit position.

In [2]:
from typing import List


def build_matrix(length: int) -> List[List[int]]:
    matrix = [[0, 0] for idx in range(0, length)]
    return matrix

sample_matrix = build_matrix(5)
print(sample_matrix)

[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]


I'll probably also want a function that can take the matrix and the line of input and increment all the counters.

In [3]:
def count_bits(matrix: List[List[int]], line: str) -> List[List[int]]:
    for i in range(0, len(line)):
        if line[i] == '0':
            matrix[i][0] += 1
        else:
            matrix[i][1] += 1
    return matrix

sample_counts = count_bits(sample_matrix, '00100')
print(sample_counts)

[[1, 0], [1, 0], [0, 1], [1, 0], [1, 0]]


Next I'll need a function to build the string with the most common bits by getting the position of the maximum of each bit counter.

In [4]:
def calc_gamma(matrix: List[List[int]]) -> str:
    gamma = ''
    for bit_counts in matrix:
        gamma += str(bit_counts.index(max(bit_counts)))
    return int(str(gamma), 2)

sample_gamma = calc_gamma(sample_matrix)
print(sample_gamma)

4


Now I think we can string it all together.  I'll need to build the appropriate sized matrix based on the length of the first line and then count the bits for each line.  Finally I'll need to calculate the gamma.  I'll also need the bit length to calculate epsilon.

In [5]:
def get_gamma(filename: str) -> int:
    with open(filename, 'r') as f:
        head = next(f).strip()
        matrix = build_matrix(len(head))
        matrix = count_bits(matrix, head)
        for line in f:
            matrix = count_bits(matrix, line.strip())
    return (calc_gamma(matrix), len(head))


sample_result = get_gamma('sample.txt')
print(sample_result)

(22, 5)


Now that I have gamma, I should be able to calculate epsilon from that.  There's probably a fancy way to do that using XOR or something, but I'm not familiar with it.  So this iteration just gets a function that converts the decimal into binary, reverses the bits, and then converts back to decimal.  Ugly, but effective.  I need the bit length to ensure that the if there are leading zeros in the binary representation of gamma that they are recorded properly.  If they are stripped by the formatting, then the conversion will produce an incorrect value due to the reduce number of bits to convert.

In [6]:
def get_epsilon(gamma: int, bits: int) -> int:
    gamma_bit_string = format(gamma, F"0{bits}b")
    epsilon_bit_string = ''
    for bit in gamma_bit_string:
        if bit == '0':
            epsilon_bit_string += '1'
        else:
            epsilon_bit_string += '0'
    return int(epsilon_bit_string, 2)

print(get_epsilon(*sample_result))

9


I think we have all the pieces.  We can test using the sample and then get the puzzle answer if everything looks good.

In [7]:
sample_gamma = get_gamma('sample.txt')
sample_epsilon = get_epsilon(*sample_gamma)
sample_answer = sample_gamma[0] * sample_epsilon
print(f"The sample gamma, {sample_gamma[0]}, multiplied by the sample epsilon, {sample_epsilon}, equals {sample_answer}")
puzzle_gamma = get_gamma('input.txt')
puzzle_epsilon = get_epsilon(*puzzle_gamma)
puzzle_answer = puzzle_gamma[0] * puzzle_epsilon
print(f"The puzzle gamma, {puzzle_gamma[0]}, multiplied by the puzzle epsilon, {puzzle_epsilon}, equals {puzzle_answer}")


The sample gamma, 22, multiplied by the sample epsilon, 9, equals 198
The puzzle gamma, 1565, multiplied by the puzzle epsilon, 2530, equals 3959450


--- Part Two ---

Next, you should verify the **life support rating**, which can be determined by multiplying the **oxygen generator rating** by the **CO2 scrubber rating**.

Both the oxygen generator rating and the CO2 scrubber rating are values that can be found in your diagnostic report - finding them is the tricky part. Both values are located using a similar process that involves filtering out values until only one remains. Before searching for either rating value, start with the full list of binary numbers from your diagnostic report and **consider just the first bit** of those numbers. Then:

* Keep only numbers selected by the **bit criteria** for the type of rating value for which you are searching. Discard numbers which do not match the bit criteria.
* If you only have one number left, stop; this is the rating value for which you are searching.
* Otherwise, repeat the process, considering the next bit to the right.

The **bit criteria** depends on which type of rating value you want to find:

* To find **oxygen generator rating**, determine the most **common value** (`0` or `1`) in the current bit position, and keep only numbers with that bit in that position. If `0` and `1` are equally common, keep values with a `1` in the position being considered.
* To find **CO2 scrubber rating**, determine the **least common value** (`0` or `1`) in the current bit position, and keep only numbers with that bit in that position. If `0` and `1` are equally common, keep values with a `0` in the position being considered.

For example, to determine the **oxygen generator rating** value using the same example diagnostic report from above:

* Start with all 12 numbers and consider only the first bit of each number. There are more `1` bits (7) than `0` bits (5), so keep only the 7 numbers with a `1` in the first position: `11110`, `10110`, `10111`, `10101`, `11100`, `10000`, and `11001`.
* Then, consider the second bit of the 7 remaining numbers: there are more `0` bits (4) than `1` bits (3), so keep only the 4 numbers with a `0` in the second position: `10110`, `10111`, `10101`, and `10000`.
* In the third position, three of the four numbers have a `1`, so keep those three: `10110`, `10111`, and `10101`.
* In the fourth position, two of the three numbers have a `1`, so keep those two: `10110` and `10111`.
* In the fifth position, there are an equal number of `0` bits and `1` bits (one each). So, to find the *oxygen generator rating*, keep the number with a `1` in that position: `10111`.
* As there is only one number left, stop; the **oxygen generator rating** is `10111`, or `23` in decimal.

Then, to determine the **CO2 scrubber rating** value from the same example above:

* Start again with all 12 numbers and consider only the first bit of each number. There are fewer `0` bits (5) than `1` bits (7), so keep only the 5 numbers with a `0` in the first position: `00100`, `01111`, `00111`, `00010`, and `01010`.
* Then, consider the second bit of the 5 remaining numbers: there are fewer `1` bits (2) than `0` bits (3), so keep only the 2 numbers with a `1` in the second position: `01111` and `01010`.
* In the third position, there are an equal number of `0` bits and `1` bits (one each). So, to find the **CO2 scrubber rating**, keep the number with a `0` in that position: `01010`.
* As there is only one number left, stop; the **CO2 scrubber rating** is `01010`, or `10` in decimal.

Finally, to find the life support rating, multiply the oxygen generator rating (`23`) by the CO2 scrubber rating (`10`) to get `230`.

Use the binary numbers in your diagnostic report to calculate the oxygen generator rating and CO2 scrubber rating, then multiply them together. **What is the life support rating of the submarine?** (Be sure to represent your answer in decimal, not binary.)


## Solution

I'll start by brute forcing this...  The **oxygen generator rating** uses the gamma value to filter down to the number.  The **CO2 scrubber rating** uses the epsilon value to filter.  I'll "cheat" by using the bits I already know for the number of bits without recalculating them.

In [8]:
def get_rating(filename: str, rating: str, bits: int) -> int:
    with open(filename, 'r') as f:
        lines = [line.strip() for line in f]
    for i in range(0, len(lines[0])):
        matrix = build_matrix(bits)
        for line in lines:
            matrix = count_bits(matrix, line)
        if matrix[i][0] == matrix[i][1]:
            if rating == 'og':
                bit = '1'
            else:
                bit = '0'
        else:
            if rating == 'og':
                bit = str(matrix[i].index(max(matrix[i])))
            else:
                bit = str(matrix[i].index(min(matrix[i])))
        lines = [line for line in lines if line[i] == bit]
        if len(lines) == 1:
            break
    return lines[0]


sample_og = int(get_rating('sample.txt', 'og', 5), 2)
sample_co2s = int(get_rating('sample.txt', 'co2s', 5), 2)

print(f"Oxygen Generator Rating: {sample_og}, CO2 Scrubber Rating: {sample_co2s}, Life Support Rating: {sample_og * sample_co2s}")

input_og = int(get_rating('input.txt', 'og', 12), 2)
input_co2s = int(get_rating('input.txt', 'co2s', 12), 2)

print(f"Oxygen Generator Rating: {input_og}, CO2 Scrubber Rating: {input_co2s}, Life Support Rating: {input_og * input_co2s}")



Oxygen Generator Rating: 23, CO2 Scrubber Rating: 10, Life Support Rating: 230
Oxygen Generator Rating: 2039, CO2 Scrubber Rating: 3649, Life Support Rating: 7440311
